In [1]:
import numpy as np
import torch
from torch.nn.functional import mse_loss
from torch.utils.data import DataLoader, WeightedRandomSampler, TensorDataset
from tqdm import tqdm
from imblearn.under_sampling import ClusterCentroids
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression

# Loading the data

In [2]:
X_train_emb = np.load('X_train_emb.npy')
y_train = np.load('y_train.npy')

In [3]:
X_train_emb.shape

(5000, 1536)

# Training the meta-model

In [4]:
low_indices = np.where(y_train <= 7)[0]
high_indices = np.where(y_train >= 7)[0]
X_train_low = X_train_emb[low_indices]
y_train_low = y_train[low_indices]
X_train_high = X_train_emb[high_indices]
y_train_high = y_train[high_indices]

In [5]:
y_bin = y_train > 7
X_resampled, y_bin_resampled = SMOTE().fit_resample(X_train_emb, y_bin)
meta_model = LogisticRegression()
meta_model.fit(X_resampled, y_bin_resampled)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


# Neural Networks

In [ ]:
#Network architecture
class SiameseNetwork(torch.nn.Module):
    def __init__(self, D, layer_1, layer_2):
        super(SiameseNetwork, self).__init__()
        self.D = D
        self.metric_fc1 = torch.nn.Linear(D, layer_1)
        self.response_fc1 = torch.nn.Linear(D, layer_1)

        self.metric_fc2 = torch.nn.Linear(layer_1, layer_2)
        self.response_fc2 = torch.nn.Linear(layer_1, layer_2)
        self.metric_enc = torch.nn.Sequential(
            self.metric_fc1,
            torch.nn.ReLU(),
            self.metric_fc2,
        )
        self.response_enc = torch.nn.Sequential(
            self.response_fc1,
            torch.nn.ReLU(),
            self.response_fc2,
        )
        self.final_lin = torch.nn.Linear(1, 1)
        


    def forward(self, x):
        metric_emb = x[:, :self.D]
        prompt_response_emb = x[:, self.D:]
        metric_out = self.metric_enc(metric_emb)
        response_out = self.response_enc(prompt_response_emb)
        #print(metric_out.shape, response_out.shape)
        #print((torch.linalg.vector_norm(metric_out, dim=1) * torch.linalg.vector_norm(response_out, dim=1)).shape)
        similarity = torch.diag(metric_out @ response_out.T) / (torch.linalg.vector_norm(metric_out, dim=1) * torch.linalg.vector_norm(response_out, dim=1))

        #print(similarity.shape)
        return self.final_lin(similarity.unsqueeze(1))

    


In [ ]:
#Separate models for low and high scores
dataset_low = TensorDataset(torch.tensor(X_train_low, dtype=torch.float32), torch.tensor(y_train_low, dtype=torch.float32))
dataset_high = TensorDataset(torch.tensor(X_train_high, dtype=torch.float32), torch.tensor(y_train_high, dtype=torch.float32))
dataloader_low = DataLoader(dataset_low, batch_size=8, shuffle=True)
dataloader_high = DataLoader(dataset_high, batch_size=64, shuffle=True)

model_low = SiameseNetwork(D=X_train_emb.shape[1]//2, layer_1=64, layer_2=16)
model_high = SiameseNetwork(D=X_train_emb.shape[1]//2, layer_1=128, layer_2=64)

In [12]:
num_epochs = 10
optimizer = torch.optim.Adam(model_low.parameters(), lr=0.1)
for epoch in range(num_epochs):
    model_low.train()
    total_loss = 0
    for X_batch, y_batch in dataloader_low:
        optimizer.zero_grad()
        y_pred = model_low(X_batch)
        loss = mse_loss(y_pred.squeeze(), y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    train_loss = total_loss / len(dataloader_low)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {train_loss:.4f}")

Epoch 1/10, Loss: 0.2501
Epoch 2/10, Loss: 0.4708
Epoch 3/10, Loss: 0.4515
Epoch 4/10, Loss: 0.3487
Epoch 5/10, Loss: 0.2406
Epoch 6/10, Loss: 0.1867
Epoch 7/10, Loss: 0.1542
Epoch 8/10, Loss: 0.1218
Epoch 9/10, Loss: 0.0746
Epoch 10/10, Loss: 0.0533
Epoch 6/10, Loss: 0.1867
Epoch 7/10, Loss: 0.1542
Epoch 8/10, Loss: 0.1218
Epoch 9/10, Loss: 0.0746
Epoch 10/10, Loss: 0.0533


In [14]:
num_epochs = 10
optimizer = torch.optim.Adam(model_high.parameters(), lr=0.1)
for epoch in range(num_epochs):
    model_high.train()
    total_loss = 0
    for X_batch, y_batch in dataloader_high:
        optimizer.zero_grad()
        y_pred = model_high(X_batch)
        loss = mse_loss(y_pred.squeeze(), y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    train_loss = total_loss / len(dataloader_high)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {train_loss:.4f}")

Epoch 1/10, Loss: 0.3055
Epoch 2/10, Loss: 0.2870
Epoch 2/10, Loss: 0.2870
Epoch 3/10, Loss: 0.2598
Epoch 3/10, Loss: 0.2598
Epoch 4/10, Loss: 0.2570
Epoch 4/10, Loss: 0.2570
Epoch 5/10, Loss: 0.2278
Epoch 5/10, Loss: 0.2278
Epoch 6/10, Loss: 0.2185
Epoch 6/10, Loss: 0.2185
Epoch 7/10, Loss: 0.2085
Epoch 7/10, Loss: 0.2085
Epoch 8/10, Loss: 0.1962
Epoch 8/10, Loss: 0.1962
Epoch 9/10, Loss: 0.1874
Epoch 9/10, Loss: 0.1874
Epoch 10/10, Loss: 0.1704
Epoch 10/10, Loss: 0.1704


In [15]:
def predict(X):
    pred_low = model_low(X)
    pred_high = model_high(X)
    probs = torch.tensor(meta_model.predict_proba(X), dtype=torch.float32)
    y_pred = (probs[:, 0]).flatten()*pred_low.flatten() + (probs[:, 1]).flatten()*pred_high.flatten()
    print(y_pred.shape)
    return torch.clip(y_pred, 0.0, 10.0)    

In [80]:
meta_model.predict_proba(X_train_emb)

array([[0.07227494, 0.92772506],
       [0.16947596, 0.83052404],
       [0.06374521, 0.93625479],
       ...,
       [0.59204161, 0.40795839],
       [0.0665085 , 0.9334915 ],
       [0.21870002, 0.78129998]], shape=(5000, 2))

In [16]:
y_train_pred = predict(torch.tensor(X_train_emb, dtype=torch.float32))
y_train = torch.tensor(y_train, dtype=torch.float32)
print(torch.sqrt(mse_loss(y_train, y_train_pred)))

torch.Size([5000])
tensor(1.1240, grad_fn=<SqrtBackward0>)


In [17]:
x_test_emb = np.load('X_test_emb.npy')
y_test_pred = predict(torch.tensor(x_test_emb, dtype=torch.float32)).detach().numpy()


torch.Size([3638])


In [18]:
y_test_pred_int = np.round(y_test_pred).astype(int)

In [57]:
with open('siamese_two_model_int_2.csv', 'w') as f:
    f.write('ID,score\n')
    for i, pred in enumerate(y_test_pred_int):
        f.write(f'{i+1},{pred}\n')

In [58]:
with open('siamese_two_model_float_2.csv', 'w') as f:
    f.write('ID,score\n')
    for i, pred in enumerate(y_test_pred):
        f.write(f'{i+1},{pred}\n')